*MeNu GUIDE*

# Matching FooDB and HMDB Compounds

In [27]:
import pandas as pd

In [ ]:
processed_data_folder = "/path/to/processed/data/folder/"

In [28]:
foodb_df = pd.read_csv(f'{processed_data_folder}foodb_compounds_with_external_descriptors.csv', dtype={'description': 'string', 'cas_number': 'string', 'kingdom': 'string', 'superclass': 'string', 'class': 'string', 'subclass': 'string', 'chebi': 'string', 'kegg': 'string','lipid_maps': 'string', 'meta_cyc': 'string', 'synonym': 'string'})

hmdb_df = pd.read_csv(f'{processed_data_folder}hmdb_metabolites.csv', dtype={'cas_registry_number': 'string', 'drugbank_id': 'string', 'knapsack_id': 'string', 'kegg_id': 'string', 'biocyc_id': 'string', 'vmh_id': 'string', 'pdb_id': 'string', 'chebi_id': 'string'})

## Merge FooDB and HMDB
### Rename identifier columns so they match.

In [29]:
hmdb_df.columns

Index(['accession', 'name', 'description', 'chemical_formula',
       'monisotopic_molecular_weight', 'iupac_name', 'cas_registry_number',
       'smiles', 'inchi', 'inchikey', 'chemspider_id', 'drugbank_id',
       'foodb_id', 'pubchem_compound_id', 'chebi_id', 'knapsack_id', 'kegg_id',
       'wikipedia_id', 'metlin_id', 'biocyc_id', 'bigg_id', 'vmh_id',
       'phenol_explorer_compound_id', 'pdb_id'],
      dtype='object')

In [30]:
foodb_df.columns

Index(['id', 'public_id', 'name', 'description', 'cas_number', 'smiles',
       'inchi', 'mono_mass', 'inchikey', 'iupac', 'kingdom', 'superclass',
       'class', 'subclass', 'chebi', 'kegg', 'lipid_maps', 'meta_cyc',
       'synonym'],
      dtype='object')

In [31]:
hmdb_df = hmdb_df.rename(columns={'accession': 'hmdb_id', 'monisotopic_molecular_weight': 'mono_mass', 'iupac_name': 'iupac', 'cas_registry_number': 'cas_number'}) 

In [32]:
foodb_df = foodb_df.rename(columns={'id': 'foodb_id_internal', 'public_id': 'foodb_id', 'chebi': 'chebi_id', 'kegg': 'kegg_id'})

## Add names in lowercase and check merging results for using only foodb_id, only the name or both

In [33]:
hmdb_df['name'] = hmdb_df['name'].str.lower()
foodb_df['name'] = foodb_df['name'].str.lower()

In [34]:
foodb_hmdb_merge = hmdb_df.merge(foodb_df, how='outer', on='foodb_id', suffixes=['_hmdb', '_foodb'])
foodb_hmdb_merge_with_name = hmdb_df.merge(foodb_df, how='outer', on=['foodb_id', 'name'], suffixes=['_hmdb', '_foodb'])
foodb_hmdb_merge_only_name = hmdb_df.merge(foodb_df, how='outer', on=['name'], suffixes=['_hmdb', '_foodb'])

### Merged Data Analysis
Have a closer look at final table to see how well the matching worked and if there are any duplicates.

In [35]:
print(f'Count of HMDB IDs: {foodb_hmdb_merge.hmdb_id.count()}')
print(f'Count of unique HMDB IDs: {foodb_hmdb_merge.hmdb_id.nunique()}\n')

print(f'Count of FooDB IDs: {foodb_hmdb_merge.foodb_id_internal.count()}')
print(f'Count of unique FooDB IDs: {foodb_hmdb_merge.foodb_id_internal.nunique()}\n')

print(f'Number of compounds that could be matched between FooDB and HMDB: {foodb_hmdb_merge[foodb_hmdb_merge.foodb_id_internal.notna()].hmdb_id.count()}')
print(f'Number of HMDB compounds that could not be matched to FooDB: {foodb_hmdb_merge[foodb_hmdb_merge.foodb_id_internal.isna()].hmdb_id.count()}')
print(f'Number of FooDB compounds that could not be matched to HMDB: {foodb_hmdb_merge[foodb_hmdb_merge.hmdb_id.isna()].foodb_id.count()}\n')

print(f'Count of duplicate matches: {foodb_hmdb_merge.foodb_id_internal.count()-foodb_hmdb_merge.foodb_id_internal.nunique()}')

Count of HMDB IDs: 217916
Count of unique HMDB IDs: 217916

Count of FooDB IDs: 70922
Count of unique FooDB IDs: 70477

Number of compounds that could be matched between FooDB and HMDB: 48374
Number of HMDB compounds that could not be matched to FooDB: 169542
Number of FooDB compounds that could not be matched to HMDB: 22548

Count of duplicate matches: 445


In [36]:
print(f'Count of HMDB IDs: {foodb_hmdb_merge_with_name.hmdb_id.count()}')
print(f'Count of unique HMDB IDs: {foodb_hmdb_merge_with_name.hmdb_id.nunique()}\n')

print(f'Count of FooDB IDs: {foodb_hmdb_merge_with_name.foodb_id_internal.count()}')
print(f'Count of unique FooDB IDs: {foodb_hmdb_merge_with_name.foodb_id_internal.nunique()}\n')

print(f'Number of compounds that could be matched between FooDB and HMDB: {foodb_hmdb_merge_with_name[foodb_hmdb_merge_with_name.foodb_id_internal.notna()].hmdb_id.count()}')
print(f'Number of HMDB compounds that could not be matched to FooDB: {foodb_hmdb_merge_with_name[foodb_hmdb_merge_with_name.foodb_id_internal.isna()].hmdb_id.count()}')
print(f'Number of FooDB compounds that could not be matched to HMDB: {foodb_hmdb_merge_with_name[foodb_hmdb_merge_with_name.hmdb_id.isna()].foodb_id_internal.count()}\n')

print(f'Count of duplicate matches: {foodb_hmdb_merge_with_name.foodb_id_internal.count()-foodb_hmdb_merge_with_name.foodb_id_internal.nunique()}')

Count of HMDB IDs: 217916
Count of unique HMDB IDs: 217916

Count of FooDB IDs: 70477
Count of unique FooDB IDs: 70477

Number of compounds that could be matched between FooDB and HMDB: 45514
Number of HMDB compounds that could not be matched to FooDB: 172402
Number of FooDB compounds that could not be matched to HMDB: 24963

Count of duplicate matches: 0


In [37]:
print(f'Count of HMDB IDs: {foodb_hmdb_merge_only_name.hmdb_id.count()}')
print(f'Count of unique HMDB IDs: {foodb_hmdb_merge_only_name.hmdb_id.nunique()}\n')

print(f'Count of FooDB IDs: {foodb_hmdb_merge_only_name.foodb_id_internal.count()}')
print(f'Count of unique FooDB IDs: {foodb_hmdb_merge_only_name.foodb_id_internal.nunique()}\n')

print(f'Number of compounds that could be matched between FooDB and HMDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.foodb_id_internal.notna()].hmdb_id.count()}')
print(f'Number of HMDB compounds that could not be matched to FooDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.foodb_id_internal.isna()].hmdb_id.count()}')
print(f'Number of FooDB compounds that could not be matched to HMDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.hmdb_id.isna()].foodb_id_internal.count()}\n')

print(f'Count of duplicate matches: {foodb_hmdb_merge_only_name.foodb_id_internal.count()-foodb_hmdb_merge_only_name.foodb_id_internal.nunique()}')

Count of HMDB IDs: 217916
Count of unique HMDB IDs: 217916

Count of FooDB IDs: 70477
Count of unique FooDB IDs: 70477

Number of compounds that could be matched between FooDB and HMDB: 63468
Number of HMDB compounds that could not be matched to FooDB: 154448
Number of FooDB compounds that could not be matched to HMDB: 7009

Count of duplicate matches: 0


In [38]:
# Check out duplicate matches for merging only via foodb_id. Seems like maybe there sometimes is an incorrect foodb_id saved for some HMDB entries
duplicate_matches = list(foodb_hmdb_merge[(foodb_hmdb_merge.foodb_id_internal.notna()) & (foodb_hmdb_merge.duplicated(subset='foodb_id_internal'))]['foodb_id'].unique())

foodb_hmdb_merge[foodb_hmdb_merge.foodb_id.isin(duplicate_matches)][['name_hmdb', 'name_foodb', 'hmdb_id', 'foodb_id', 'chebi_id_hmdb', 'chebi_id_foodb']]

,name_hmdb,name_foodb,hmdb_id,foodb_id,chebi_id_hmdb,chebi_id_foodb
281,l-tyrosine,l-tyrosine,HMDB0000158,FDB000446,17895,CHEBI:17895
282,d-tyrosine,l-tyrosine,HMDB0250803,FDB000446,18186,CHEBI:17895
427,brassicoside,isorhamnetin 3-sophoroside 7-glucoside,HMDB0029480,FDB000607,<NA>,<NA>
428,carboxyyessotoxin,isorhamnetin 3-sophoroside 7-glucoside,HMDB0036622,FDB000607,<NA>,<NA>
434,isotheaflavin,isotheaflavin,HMDB0029483,FDB000613,<NA>,<NA>
...,...,...,...,...,...,...
24794,(cis)-crotonaldehyde,(cis)-crotonaldehyde,HMDB0303986,FDB030138,<NA>,<NA>
25559,oxalosuccinic acid,oxalosuccinate,HMDB0003974,FDB031076,44712,<NA>
25560,oxalosuccinate,oxalosuccinate,HMDB0304444,FDB031076,7815,<NA>
28967,rac-4-hydroxy-4-o-(beta-d-glucuronide)-all-tra...,rac-4-hydroxy-4-o-(beta-d-glucuronide)-all-tra...,HMDB0060141,FDB034577,<NA>,<NA>


In [39]:
# Check if there are any inconsistencies in foodb_ids after matching only via name
foodb_hmdb_merge_only_name[(foodb_hmdb_merge_only_name.foodb_id_hmdb != foodb_hmdb_merge_only_name.foodb_id_foodb) & (foodb_hmdb_merge_only_name.foodb_id_hmdb.notna()) & (foodb_hmdb_merge_only_name.foodb_id_foodb.notna())][['name', 'foodb_id_foodb', 'foodb_id_hmdb']]

,name,foodb_id_foodb,foodb_id_hmdb
19,(+)-10-methyldodecanoic acid,FDB098155,FDB098154
36,(+)-7-iso-jasmonate,FDB030070,FDB030077
40,(+)-alpha-carene,FDB013230,FDB013714
88,(+)-hardwickiic acid,FDB006871,FDB006884
117,(+)-norushinsunine n-oxide,FDB012978,FDB015501
...,...,...,...
224716,yuccoside c,FDB012029,FDB020816
224774,zeaxanthin,FDB014726,FDB023113
224776,zederone,FDB015708,FDB017170
224802,zinc,FDB003729,FDB031256


After comparing the entries on the respective websites, it seems it makes the most amount of sense to merge based on name. This yields the most hits, while also having no duplicates. Now we need to merge all the identifiers. It makes sense that the foodb_id should be preferred, but if there is none, we should tak the foodb_id provided by HMDB.

In [40]:
foodb_hmdb_merge_only_name.loc[:, 'foodb_id'] = foodb_hmdb_merge_only_name.apply(lambda row: row.foodb_id_foodb if pd.notna(row.foodb_id_foodb) else None, axis=1)
foodb_hmdb_merge_only_name = foodb_hmdb_merge_only_name.drop(columns=['foodb_id_hmdb', 'foodb_id_foodb'])

In [41]:
print(f'Count of HMDB IDs: {foodb_hmdb_merge_only_name.hmdb_id.count()}')
print(f'Count of unique HMDB IDs: {foodb_hmdb_merge_only_name.hmdb_id.nunique()}\n')

print(f'Count of FooDB IDs: {foodb_hmdb_merge_only_name.foodb_id.count()}')
print(f'Count of unique FooDB IDs: {foodb_hmdb_merge_only_name.foodb_id.nunique()}\n')

print(f'Number of compounds that could be matched between FooDB and HMDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.foodb_id_internal.notna()].hmdb_id.count()}')
print(f'Number of HMDB compounds that could not be matched to FooDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.foodb_id_internal.isna()].hmdb_id.count()}')
print(f'Number of FooDB compounds that could not be matched to HMDB: {foodb_hmdb_merge_only_name[foodb_hmdb_merge_only_name.hmdb_id.isna()].foodb_id_internal.count()}\n')

print(f'Count of duplicate matches: {foodb_hmdb_merge_only_name.foodb_id_internal.count()-foodb_hmdb_merge_only_name.foodb_id_internal.nunique()}')

Count of HMDB IDs: 217916
Count of unique HMDB IDs: 217916

Count of FooDB IDs: 70477
Count of unique FooDB IDs: 70477

Number of compounds that could be matched between FooDB and HMDB: 63468
Number of HMDB compounds that could not be matched to FooDB: 154448
Number of FooDB compounds that could not be matched to HMDB: 7009

Count of duplicate matches: 0


## Clean up columns of merged data

In [169]:
foodb_hmdb_merge_only_name.columns

Index(['hmdb_id', 'name', 'description_hmdb', 'chemical_formula',
       'mono_mass_hmdb', 'iupac_hmdb', 'cas_number_hmdb', 'smiles_hmdb',
       'inchi_hmdb', 'inchikey_hmdb', 'chemspider_id', 'drugbank_id',
       'pubchem_compound_id', 'chebi_id_hmdb', 'knapsack_id', 'kegg_id_hmdb',
       'wikipedia_id', 'metlin_id', 'biocyc_id', 'bigg_id', 'vmh_id',
       'phenol_explorer_compound_id', 'pdb_id', 'foodb_id_internal',
       'description_foodb', 'cas_number_foodb', 'smiles_foodb', 'inchi_foodb',
       'mono_mass_foodb', 'inchikey_foodb', 'iupac_foodb', 'kingdom',
       'superclass', 'class', 'subclass', 'chebi_id_foodb', 'kegg_id_foodb',
       'lipid_maps', 'meta_cyc', 'synonym', 'foodb_id'],
      dtype='object')

In [170]:
# Merge descriptions. Strategy: HMDB seems to be correct more often than FooDB, so go with HMDB info, unless there is only FooDB info available. 
columns_to_clean = ["description", "mono_mass", "iupac", "inchi", "inchikey", "cas_number", "smiles", "chebi_id", "kegg_id"]

for column in columns_to_clean:
    column_foodb = column + '_foodb'
    column_hmdb = column + '_hmdb'
    
    foodb_hmdb_merge_only_name[column] = foodb_hmdb_merge_only_name.apply(lambda row: row[column_hmdb] if pd.notna(row[column_hmdb]) else row[column_foodb], axis=1)
    foodb_hmdb_merge_only_name = foodb_hmdb_merge_only_name.drop(columns=[column_foodb, column_hmdb])
    
    if column == 'chebi_id':
        foodb_hmdb_merge_only_name[column] = foodb_hmdb_merge_only_name[column].apply(lambda x: x.strip('CHEBI:') if pd.notna(x) else x)

In [171]:
foodb_hmdb_merge_only_name.columns

Index(['hmdb_id', 'name', 'chemical_formula', 'chemspider_id', 'drugbank_id',
       'pubchem_compound_id', 'knapsack_id', 'wikipedia_id', 'metlin_id',
       'biocyc_id', 'bigg_id', 'vmh_id', 'phenol_explorer_compound_id',
       'pdb_id', 'foodb_id_internal', 'kingdom', 'superclass', 'class',
       'subclass', 'lipid_maps', 'meta_cyc', 'synonym', 'foodb_id',
       'description', 'mono_mass', 'iupac', 'inchi', 'inchikey', 'cas_number',
       'smiles', 'chebi_id', 'kegg_id'],
      dtype='object')

In [173]:
foodb_hmdb_merge_only_name.to_csv(f'{processed_data_folder}/foodb_hmdb.csv', index=False)

## Conclusion:
* Match was most successful if using only the name cast to lower case
* Various problems are likely due to incorrect entries in HMDB and the fact that the FooDB version that can be downloaded from the website is not up to date, which is probably why HMDB has a higher count of unique FooDB IDs than FooDB
* The process has been adjusted so that there are no duplicate matches